In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.linear_model import LinearRegression

In [ ]:
df = pd.read_csv('CGMData/03-05-2024-03-18-2024.csv')
display(df)

In [ ]:
df = df.drop(columns=['SerialNumber', 'Description', 'DeviceType'])
display(df)

In [ ]:
# Convert the 'DateTime' column to datetime format
df['DateTime'] = pd.to_datetime(df['EventDateTime'], format='%m/%d/%Y %H:%M')

# Extract hour and minute features
df['Hour'] = df['DateTime'].dt.hour
df['Minute'] = df['DateTime'].dt.minute

df = df.drop(columns=['EventDateTime'])

display(df)

In [ ]:
df = df.dropna()

last_3_days_data = df[df['DateTime'] >= df['DateTime'].max() - timedelta(days=3)]

# Split the data into training and testing sets
X = last_3_days_data[['Hour', 'Minute']]
y = last_3_days_data['Readings (CGM / BGM)']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Train the RandomForestRegression model
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)

# Make predictions for 30 minutes into the future
predictions = rf.predict(X_test)

# Calculate and print the Mean Squared Error (MSE)
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

# Example prediction for the next 30 minutes
next_hour = df['Hour'].iloc[-1]  # Assuming the last entry in the DataFrame is the latest hour
next_minute = df['Minute'].iloc[-1] + 30  # Adding 30 minutes to the latest minute
if next_minute >= 60:
    next_hour += 1
    next_minute -= 60
prediction_input = np.array([[next_hour, next_minute]])
next_prediction = rf.predict(prediction_input)
print(f'Predicted glucose reading for {next_hour:02d}:{next_minute:02d}: {next_prediction[0]}')

In [ ]:
# Filter the DataFrame to include only the data for the last 3 days
last_3_days_data = df[df['DateTime'] >= df['DateTime'].max() - timedelta(days=3)]

# Make predictions for the last 3 days
#predictions = rf.predict(last_3_days_data[['Hour', 'Minute']])

# Plot the actual and predicted glucose readings for the last 3 days
plt.figure(figsize=(12, 6))
plt.plot(last_3_days_data['DateTime'], last_3_days_data['Readings (CGM / BGM)'], label='Actual Readings')
plt.plot(last_3_days_data['DateTime'], predictions, label='Predicted Readings', linestyle='--')
plt.xlabel('Date and Time')
plt.ylabel('Glucose Readings')
plt.title('Actual vs. Predicted Glucose Readings for Last 3 Days')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()